## [Hire K workers with minimum cost](https://leetcode.com/problems/minimum-cost-to-hire-k-workers/)
- N workers:
    + quality[i]: worker quality
    + wage[i]: worker minimum expected pays
- Rules: Hire K out of N worker with minimum total cost
    + 1. Hired woker should be paid in the ratio of their quality compared to other workers
    + 2. Every hired worker must be paid at least their minimum wage expectation.

```
Example 1
    Input: quality = [10,20,5], wage = [70,50,30], K = 2
    Output: 105.00000
    Explanation: We pay 70 to 0-th worker and 35 to 2-th worker.
    
Example 2
    Input: quality = [3,1,10,10,1], wage = [4,8,2,2,7], K = 3
    Output: 30.66667
    Explanation: We pay 4 to 0-th worker, 13.33333 to 2-th and 3-th workers 
```

#### Solution - Bruteforce $O(N^2log(N))$
- Select a captain c that choose his ratio as the base salary
- Loop through all possible options and choose the min_cost

```C++
double mincostToHireWorkers(vector<int>& quality, vector<int>& wage, int K) {
    int n = quality.size();
    double min_cost = (double)(1e9 + 7);
    double total_wages;

    double ratio;
    vector<double> worker_wages;

    for(int c=0; c<n; ++c) {
        // select a captain -> choose his ratio to compare others
        ratio = (double)wage[c] / quality[c];
        total_wages = (double)wage[c];

        // Based on the captain ratio add the worker pays
        worker_wages.clear();
        for(int w=0; w<n; ++w) {
            // Select workers that not the captain and must pay > their expected 
            if(w==c || ratio*quality[w] < wage[w]) continue;
            worker_wages.push_back(ratio*quality[w]);
        }

        // Prune the unsatisfied solutions
        if(worker_wages.size() < K-1) continue;

        // Get the result 
        sort(worker_wages.begin(), worker_wages.end());
        for(int w=0; w<K-1; ++w)
            total_wages += worker_wages[w];
        min_cost = min(min_cost, total_wages);
    }
    return min_cost;
}
```

#### Solution - $O(Nlog(N))$

<img src="./img/1.jpg" alt="drawing" width="1000"/>


```C++
double mincostToHireWorkers(vector<int>& quality, vector<int>& wage, int K) {
    int n = quality.size();
    double min_cost = numeric_limits<double>::max();
    double total_qual = 0.0;
    double base_ratio;

    // Sort the worker by increasing ratio = wage / qual 
    vector<Worker> workers;
    for(int w=0; w<n; ++w) 
        workers.push_back( {quality[w], wage[w]} );
    sort(workers.begin(), workers.end(), Less());

    // maxHeap maintain minimum total_cost at w == captain
    priority_queue<int> maxHeap;
    for(int w=0; w<n; ++w) {
        maxHeap.push(workers[w].quality);
        total_qual += workers[w].quality;

        if(maxHeap.size() > K) {
            total_qual -= maxHeap.top();
            maxHeap.pop();
        }
        if(maxHeap.size() == K) {
            base_ratio = (double)workers[w].wage / workers[w].quality; 
            min_cost = min(min_cost, base_ratio*total_qual);
        }
    }

    return min_cost;
}
```